# MediaPipe 작업으로 랜드마크 감지 포즈

이 노트북은 MediaPipe Tasks Python API를 사용하여 이미지에서 포즈 랜드마크를 감지하는 방법을 보여줍니다.

## 준비

MediaPipe 설치부터 시작해 보겠습니다.


In [ ]:
!pip install -q mediapipe==0.10.0

그런 다음 기성 모델 번들을 다운로드하세요. 이 모델 번들에 대한 자세한 내용은 [MediaPipe 문서](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker#models)를 확인하세요.

In [ ]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

## 시각화 유틸리티

In [ ]:
#@markdown Pose Landmarker API를 더 잘 보여주기 위해 우리는 이 Colab에서 사용할 시각화 도구 세트를 만들었습니다. 이를 통해 감지하는 사람의 랜드마크와 해당 마커 간의 예상 연결이 그려집니다.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

## 테스트 이미지 다운로드

Pose Landmarker API를 시연하기 위해 다음 코드를 사용하여 샘플 이미지를 다운로드할 수 있습니다. 이미지 출처는 [Pixabay](https://pixabay.com/photos/girl-woman-fitness-beautiful-smile-4051811/)입니다.

In [ ]:
!wget -q -O image.jpg https://cdn.pixabay.com/photo/2019/03/12/20/39/girl-4051811_960_720.jpg

import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("image.jpg")
cv2_imshow(img)

선택적으로 자신의 이미지를 업로드할 수 있습니다. 그렇게 하려면 아래 셀의 주석 처리를 해제하고 실행하십시오.

In [ ]:
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# if len(uploaded.keys()):
#   IMAGE_FILE = next(iter(uploaded))
#   print('Uploaded file:', IMAGE_FILE)

## 추론 실행 및 결과 시각화

마지막 단계는 선택한 이미지에서 포즈 랜드마크 감지를 실행하는 것입니다. 여기에는 PoseLandmarker 개체 생성, 이미지 로드, 감지 실행, 마지막으로 시각화를 통해 이미지를 표시하는 선택적 단계가 포함됩니다.

이 솔루션이 지원하는 구성 옵션에 대해 자세히 알아보려면 [MediaPipe 문서](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python)를 확인하세요.


In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("image.jpg")

# STEP 4: Detect pose landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

포즈 분할 마스크를 시각화합니다.

In [ ]:
segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255
cv2_imshow(visualized_mask)